## LLM
----

Across just about every domain, deploying massive deep learning models is a common yet challenging task. Today's models, such as Llama 2 (70B parameters) or ensemble models like Mixtral 7x8B, are products of advanced training methods, vast data resources, and powerful computing systems. Luckily for us, these models have already been trained and many use cases can already be achieved with off-the-shelf solutions. The real hurdle, however, lies in effectively hosting these models.

**Deployment Scenarios for Large Models:**

1. **High-End Datacenter Deployment:**
> An uncompressed, unquantized model on a data center stack equipped with GPUs like NVIDIA's [A100](https://www.nvidia.com/en-us/data-center/a100/)/[H100](https://www.nvidia.com/en-us/data-center/h100/)/[H200](https://www.nvidia.com/en-us/data-center/h200/) to facilitate fast inference and experimentation.
> - **Pros**: Ideal for scalable deployment and experimentation, this stack is ideal for either large training workflows or for supporting multiple users or models at the same time.  
> - **Cons:** It is inefficient to allocate this resource for each user of your service unless the use cases involve model training/fine-tuning or interfacing with lower-level model components.

2. **Modest Datacenter/Specialized Consumer Hardware Deployment:**
> Quantized and further-optimized models can be run (one or two per instance) on more conservative datacenter GPUs such as [L40](https://www.nvidia.com/en-us/data-center/l40/)/[A30](https://www.nvidia.com/en-us/data-center/products/a30-gpu/)/[A10](https://www.nvidia.com/en-us/data-center/products/a10-gpu/) or even on some modern consumer GPUs such as the higher-VRAM [RTX 40-series GPUs](https://www.nvidia.com/en-us/geforce/graphics-cards/40-series/).
> - **Pros:** This setup balances inference speed with manageable limitations for single-user applications. These sessions can also be deployed on a per-user basis to run one or two large models at a time with raw access to model internals (even if they need quantization).
> - **Cons:** Deploying an instance for each user is still costly at scale, though it may be justifiable for some niche workloads. Alternatively, assuming that users can access these resources in their local environments is likely unreasonable.

3. **Consumer Hardware Deployment:**
> Though heavily limited in ability to propagate data through a neural network, most consumer hardware does have a graphical user interface (GUI), a web browser with internet access, some amount of memory (can safely assume at least 1 GB), and a decently-powerful CPU.
> - **Cons:** Most hardware at the moment cannot run more than one local large model at a time in any configuration, and running even one model will require significant amounts of resource management and optimizing restrictions.
> - **Pros:** This is a reasonable and inclusive starting assumption when considering what kinds of users your services should support.


----

<br>

## Hosted Large Model Services

**Black-Box Hosted Models:**
> Services such as [**OpenAI**](https://openai.com/) offer APIs to interact with black-box models like GPT-4. These powerful, well-integrated services can provide simple interfaces to complex pipelines that automatically track memory, call additional models, and incorporate multimodal interfaces as necessary to simplify typical use scenarios. At the same time, they maintain operational opacity and often lack a straightforward path to self-hosting.
> - **Pros:** Easy to use out-of-the-box with shallow barriers to entry for an average user.
> - **Cons:** Black-box deployments suffer from potential privacy concerns, limited customization, and cost implications at scale.

**Self-Hosted Models:**

> Behind the scenes of just about all scaled model deployments is one or more giant models running in a data center with scalable resources and lightning-fast bandwidth at their disposal. Though necessary to deploy large models at scale and maintain strong control over the provided interfaces, these systems often require expertise to set up and generally do not work well for supporting non-developer workflows for only one individual at a time. Such systems are much better for supporting many simultaneous users, multiple models, and custom interfaces.
> - **Pros:** They offer the capability to integrate custom datasets and APIs and are primarily designed to support numerous users concurrently.
> - **Cons:** These setups demand technical expertise to set up and properly configure.

To get the best of both worlds, we will utilize the [**NVIDIA NGC Service**](https://www.nvidia.com/en-us/gpu-cloud/). NGC offers a suite of developer tools for designing and deploying AI solutions. Central to our needs are the [NVIDIA AI Foundation Models](https://www.nvidia.com/en-us/ai-data-science/foundation-models/), which are pre-tuned and pre-optimized models designed for easy out-of-the-box scalable deployment (as-is or with further customization). Furthermore, NGC hosts accessible model endpoints for querying live foundation models in a [scalable DGX-accelerated compute environment](https://www.nvidia.com/en-us/data-center/dgx-platform/).

## Getting Started With Hosted Inference
**When deploying a model for scaled inference, the steps you generally need to take are as follows:**
- Identify the models you would like users to access, and allocate resources to host them.
- Figure out what kinds of controls you would like users to have, and expose ways for them to access it.
- Create monitoring schemes to track/gate usage, and set up systems to scale and throttle as necessary.

For this, we'll use the models deployed by NVIDIA, which are hosted as **LLM NIMs.** NIMs are microservices that are optimized to run AI workloads for scaled inference deployment. They work just fine for local inference and offer standardized APIs, but are primarily designed to work especially well in scaled environments. These particular models are deployed on NVIDIA DGX Cloud as shared functions and are advertised through an OpenAPI-style API gateway. Let's unpack what that means:

**On The Cluster Side:** These microservices are hosted on a Kubernetes-backed platform that scales the load across a minimum and maximum number of DGX Nodes and are delivered behind a single function. In other words:
- A large-language model is downloaded to and deployed on a **GPU-enabled compute node** (i.e. a powerful CPU and 4xH100-GPU environment which is physically-integrated in a DGX Pod).
- On start, a selection of these compute nodes are kickstarted such that, whenever a user sends a request to the function, one of those nodes will receive the request.
    - Kubernetes will route this traffic appropriately. If there is an idle compute node, it will receive the traffic. If all of them are working, the request will be queued up and a node will pick it up as soon as possible.
    - In our case, these nodes will still pick up requests very fast since in-flight batching is enabled, meaning each node can take in up to 256 active requests at a time as-they-come before they get completely "full". (256 is a hyperparameter on deployment).
- As load begins to increase, auto-scaling will kick in and more nodes will be kickstarted to avoid request handling delays.

The following image shows an arbitrary function invocation with a custom (non-OpenAPI) API. This was the initial way in which the public endpoints were advertised, but is now an implementation detail.

<!-- > <img style="max-width: 1000px;" src="imgs/ai-playground-api.png" /> -->
<!-- > <img src="https://drive.google.com/uc?export=view&id=1ckAIZoy7tvtK1uNqzA9eV5RlKMbVqs1-" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/ai-playground-api.png" width=800px/>

**On The Gateway Side:** To make this API more standard, an API gateway server is used to aggregate these functions behind a common API known as OpenAPI. This specification is subscribed to by many including OpenAI, so using the OpenAI client is a valid interface:

<!-- > <img style="max-width: 800px;" src="imgs/mixtral_api.png" /> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/mixtral_api.png" width=800px/>

**On The User Side:** Incorporating these endpoints into your client, you can design integrations, pipelines, and user experiences that leverage these generative AI capabilities to endow your applications with reasoning and generative abilities. A popular example of such an application is [**OpenAI's ChatGPT**](https://chat.openai.com/), which is an orchestration of endpoints including GPT4, Dalle, and others. Though it may sometimes look like a single intelligent model, it is merely an aggregation of model endpoints with software engineering to help manage state and context control. This will be reinforced throughout the course, and by the end you should have an idea for how you could go about making a similar chat assistant for an arbitrary use-case.

<!-- > <img style="max-width: 700px;" src="imgs/openai_chat.png" /> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/openai_chat.png" width=700px/>


----

<br>

## Trying Out The Foundation Model Endpoints



In [2]:
import os
os.environ["NVIDIA_API_KEY"] = "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"

### ChatNVIDIA Client Request
In this experiment, we will want to do LLM orchestration with a framework called LangChain, so we'll need to go one layer of abstraction higher to a **Framework Connector**.

The goal of a **connector** is to convert an arbitrary API from its native core into one that a target code-base would expect.

Here, we'll want to take advantage of LangChain's thriving chain-centric ecosystem, but the raw `requests` API will not take us all the way there. Under the hood, every LangChain chat model that isn't hosted locally has to rely on such an API, but the developer-facing API is a much cleaner [`LLM` or `SimpleChatModel`-style interface](https://python.langchain.com/docs/modules/model_io/) with default parameters and some simple utility functions like `invoke` and `stream`.

To start off our exploration into the LangChain interface, we will use the `ChatNVIDIA` connector to interface with our `chat/completions` endpoints. This model is part of the LangChain extended ecosystem and can be installed locally via `pip install langchain-nvidia-ai-endpoints`.

In [3]:
!pip install langchain-nvidia-ai-endpoints

In [4]:
## Using ChatNVIDIA
from langchain_nvidia_ai_endpoints import ChatNVIDIA

## NVIDIA_API_KEY pulled from environment
llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")

llm.invoke("Hello!, who are you?")

AIMessage(content='Hello! I am a large language model trained by the Mistral AI team. I am designed to generate human-like text based on the input I receive. I do not have the ability to access personal data about individuals, perform external tasks, or maintain a stateful conception of myself beyond the current conversation. I am simply a tool for generating text based on the input I receive. How can I assist you today?', response_metadata={'role': 'assistant', 'content': 'Hello! I am a large language model trained by the Mistral AI team. I am designed to generate human-like text based on the input I receive. I do not have the ability to access personal data about individuals, perform external tasks, or maintain a stateful conception of myself beyond the current conversation. I am simply a tool for generating text based on the input I receive. How can I assist you today?', 'token_usage': {'prompt_tokens': 16, 'total_tokens': 101, 'completion_tokens': 85}, 'finish_reason': 'stop', 'mod

In [5]:
llm

ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='mistralai/mixtral-8x7b-instruct-v0.1')

In [6]:
model_list = ChatNVIDIA.get_available_models()

for model_card in model_list:
    model_name = model_card.id

    print(model_name)

meta/llama2-70b
writer/palmyra-med-70b-32k
nvidia/llama3-chatqa-1.5-70b
nvidia/usdcode-llama3-70b-instruct
nvidia/neva-22b
meta/llama-3.1-8b-instruct
adept/fuyu-8b
upstage/solar-10.7b-instruct
meta/llama-3.1-70b-instruct
google/deplot
nvidia/nemotron-4-340b-instruct
microsoft/phi-3-mini-4k-instruct
meta/llama-3.1-405b-instruct
seallms/seallm-7b-v2.5
mediatek/breeze-7b-instruct
ibm/granite-8b-code-instruct
nv-mistralai/mistral-nemo-12b-instruct
mistralai/codestral-22b-instruct-v0.1
google/codegemma-7b
meta/codellama-70b
liuhaotian/llava-v1.6-34b
google/gemma-2-27b-it
meta/llama3-70b-instruct
google/paligemma
liuhaotian/llava-v1.6-mistral-7b
mistralai/mistral-large
deepseek-ai/deepseek-coder-6.7b-instruct
ibm/granite-34b-code-instruct
google/recurrentgemma-2b
google/gemma-2-2b-it
microsoft/phi-3-medium-4k-instruct
mistralai/mixtral-8x22b-instruct-v0.1
google/gemma-2-9b-it
mistralai/mistral-7b-instruct-v0.2
google/codegemma-1.1-7b
mistralai/mistral-7b-instruct-v0.3
writer/palmyra-med-70b
